In [1]:
import os

def total_files(folder_path):
    num_files = len([f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))])
    return num_files

# Paths to your training, test, and validation directories
train_dir = "D:/project_1/IMPLEMENTATION/OG Dataset/train"
test_dir = "D:/project_1/IMPLEMENTATION/OG Dataset/test"
valid_dir = "D:/project_1/IMPLEMENTATION/OG Dataset/validation"
# Class names
classes = ['BLACK_SPOT', 'DOWNY_MILDEW', 'FRESH_LEAF', 'POWDERY_MILDEW', 'ROSE_MOSAIC', 'ROSE_RUST', 'ROSE_SLUG']

# Count images in training set
for class_name in classes:
    train_path = os.path.join(train_dir, class_name)
    print(f"Number of {class_name} leaf images in training set:", total_files(train_path))

print("========================================================")

# Count images in test set
for class_name in classes:
    test_path = os.path.join(test_dir, class_name)
    print(f"Number of {class_name} leaf images in test set:", total_files(test_path))

print("========================================================")

# Count images in validation set
for class_name in classes:
    valid_path = os.path.join(valid_dir, class_name)
    print(f"Number of {class_name} leaf images in validation set:", total_files(valid_path))


Number of BLACK_SPOT leaf images in training set: 700
Number of DOWNY_MILDEW leaf images in training set: 700
Number of FRESH_LEAF leaf images in training set: 700
Number of POWDERY_MILDEW leaf images in training set: 700
Number of ROSE_MOSAIC leaf images in training set: 700
Number of ROSE_RUST leaf images in training set: 700
Number of ROSE_SLUG leaf images in training set: 700
Number of BLACK_SPOT leaf images in test set: 150
Number of DOWNY_MILDEW leaf images in test set: 150
Number of FRESH_LEAF leaf images in test set: 150
Number of POWDERY_MILDEW leaf images in test set: 150
Number of ROSE_MOSAIC leaf images in test set: 150
Number of ROSE_RUST leaf images in test set: 150
Number of ROSE_SLUG leaf images in test set: 150
Number of BLACK_SPOT leaf images in validation set: 150
Number of DOWNY_MILDEW leaf images in validation set: 150
Number of FRESH_LEAF leaf images in validation set: 150
Number of POWDERY_MILDEW leaf images in validation set: 150
Number of ROSE_MOSAIC leaf image

In [2]:
# Import necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad, Adadelta


In [3]:
# dataset paths
train_dir = "D:/project_1/IMPLEMENTATION/OG Dataset/train"
test_dir = "D:/project_1/IMPLEMENTATION/OG Dataset/test"
valid_dir = "D:/project_1/IMPLEMENTATION/OG Dataset/validation"


In [4]:
disease_dict = {
    0: 'BLACK_SPOT',
    1: 'DOWNY_MILDEW',
    2: 'FRESH_LEAF',
    3: 'POWDERY_MILDEW',
    4: 'ROSE_MOSAIC',
    5: 'ROSE_RUST',
    6: 'ROSE_SLUG'
}
num_classes = len(disease_dict)


In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [6]:
train_datagen = ImageDataGenerator(rescale=1.0/255.0)

validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)


In [7]:
batch_size = 32


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),  
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = validation_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  
    batch_size=batch_size,
    class_mode='categorical'
)


Found 4900 images belonging to 7 classes.
Found 1050 images belonging to 7 classes.
Found 1050 images belonging to 7 classes.


In [12]:

output_folder = 'confusion_matrices'
os.makedirs(output_folder, exist_ok=True)

# Hyperparameters to experiment with
activations = ['relu', 'tanh', 'sigmoid']
optimizers = ['adam', 'sgd', 'rmsprop']
num_layers = 4
neurons = 128
batch_size = 16


experiment_results = []


In [13]:
def build_custom_cnn(activation, optimizer, num_layers, neurons):
    model = Sequential()
    model.add(Conv2D(16, (3, 3), activation=activation, padding='same', input_shape=(224, 224, 3)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))

    for _ in range(num_layers - 1):  # Add additional layers
        model.add(Conv2D(32, (3, 3), activation=activation, padding='same'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Flatten())
    model.add(Dense(neurons, activation=activation))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))  # Ensure num_classes is defined

    # Select optimizer
    if optimizer == 'adam':
        opt = Adam(learning_rate=1e-4)
    elif optimizer == 'sgd':
        opt = SGD(learning_rate=0.01)
    else:
        opt = RMSprop(learning_rate=0.001)

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)     │ (None, 7, 7, 1280)     │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,214,442 (16.08 MB)

 Trainable params: 164,871 (644.03 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [14]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, confusion_matrix
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import os

In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, confusion_matrix

# Paths
results_csv_path = 'experiment_results.csv'
confusion_matrix_folder = 'confusion_matrices'
os.makedirs(confusion_matrix_folder, exist_ok=True)

# Prepare the results DataFrame
columns = ['activation', 'optimizer', 'num_layers', 'neurons', 'accuracy', 'precision', 'recall', 'f1_score']
results_df = pd.DataFrame(columns=columns)


for activation in activations:
    for optimizer in optimizers:
        
        print(f"Training model with {activation}, {optimizer}, {num_layers} layers, {neurons} neurons, batch size {batch_size}")

       
        model = build_custom_cnn(activation=activation, optimizer=optimizer, num_layers=num_layers, neurons=neurons)
        
        # Train the model with callbacks
        model.fit(
            train_generator,
            epochs=7,
            validation_data=validation_generator,
            verbose=1
        )
        
        # Evaluate the model
        test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
        print(f"Accuracy of {activation}_{optimizer}_{num_layers}_{neurons} model: {test_accuracy}")

        # Get true and predicted labels
        y_true = test_generator.classes
        y_pred_probs = model.predict(test_generator)
        y_pred = np.argmax(y_pred_probs, axis=1)

        # Calculate precision, recall, and F1 score
        precision = precision_score(y_true, y_pred, average='weighted')
        recall = recall_score(y_true, y_pred, average='weighted')
        f1 = f1_score(y_true, y_pred, average='weighted')

        # Print the classification report
        report = classification_report(y_true, y_pred, output_dict=True)
        print(report)

        # Store the results in DataFrame
        results_df = pd.concat([results_df, pd.DataFrame({
            'activation': [activation],
            'optimizer': [optimizer],
            'num_layers': [num_layers],
            'neurons': [neurons],
            'accuracy': [test_accuracy],
            'precision': [precision],  
            'recall': [recall],
            'f1_score': [f1]
        })], ignore_index=True)

        # # Generate and save confusion matrix
        # cm = confusion_matrix(y_true, y_pred)
        # plt.figure(figsize=(10, 7))
        # sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
        # plt.ylabel('True label')
        # plt.xlabel('Predicted label')
        # plt.title(f'Confusion Matrix: {activation}, {optimizer}, Layers: {num_layers}, Neurons: {neurons}')
        # plt.savefig(os.path.join(confusion_matrix_folder, f'{activation}_{optimizer}_{neurons}_{num_layers}.png'))
        # plt.close()

# Save all experiment results to CSV
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")


Training model with relu, adam, 4 layers, 128 neurons, batch size 16


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 0s 511ms/step - accuracy: 0.4498 - loss: 1.6667

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


307/307 ━━━━━━━━━━━━━━━━━━━━ 182s 582ms/step - accuracy: 0.4502 - loss: 1.6652 - val_accuracy: 0.4210 - val_loss: 2.0421
Epoch 2/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 177s 571ms/step - accuracy: 0.7204 - loss: 0.7298 - val_accuracy: 0.8333 - val_loss: 0.4795
Epoch 3/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 190s 613ms/step - accuracy: 0.7934 - loss: 0.5586 - val_accuracy: 0.8571 - val_loss: 0.3579
Epoch 4/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 191s 617ms/step - accuracy: 0.8408 - loss: 0.4413 - val_accuracy: 0.8781 - val_loss: 0.2900
Epoch 5/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 117s 373ms/step - accuracy: 0.8666 - loss: 0.3601 - val_accuracy: 0.8790 - val_loss: 0.2951
Epoch 6/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 90s 289ms/step - accuracy: 0.8707 - loss: 0.3303 - val_accuracy: 0.8990 - val_loss: 0.2602
Epoch 7/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 118s 381ms/step - accuracy: 0.8965 - loss: 0.2771 - val_accuracy: 0.9114 - val_loss: 0.2388
66/66 ━━━━━━━━━━━━━━━━━━━━ 16s 233ms/step - accuracy: 0.8999 - loss: 0.2650
Accuracy of relu_a

...

Epoch 1/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 109s 342ms/step - accuracy: 0.5203 - loss: 1.2896 - val_accuracy: 0.1486 - val_loss: 2.2950
Epoch 2/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 107s 345ms/step - accuracy: 0.7332 - loss: 0.7194 - val_accuracy: 0.6933 - val_loss: 0.7776
Epoch 3/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 86s 279ms/step - accuracy: 0.7677 - loss: 0.6131 - val_accuracy: 0.8295 - val_loss: 0.4726
Epoch 4/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 103s 334ms/step - accuracy: 0.8118 - loss: 0.5168 - val_accuracy: 0.8819 - val_loss: 0.3868
Epoch 5/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 195s 633ms/step - accuracy: 0.8481 - loss: 0.4414 - val_accuracy: 0.8524 - val_loss: 0.3823
Epoch 6/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 194s 628ms/step - accuracy: 0.8579 - loss: 0.4224 - val_accuracy: 0.8610 - val_loss: 0.3650
Epoch 7/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 174s 561ms/step - accuracy: 0.8496 - loss: 0.4115 - val_accuracy: 0.8771 - val_loss: 0.3407
66/66 ━━━━━━━━━━━━━━━━━━━━ 7s 107ms/step - accuracy: 0.8852 - loss: 0.3301
Accuracy 

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 188s 604ms/step - accuracy: 0.4936 - loss: 1.3505 - val_accuracy: 0.3152 - val_loss: 1.7266
Epoch 2/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 193s 622ms/step - accuracy: 0.7054 - loss: 0.8058 - val_accuracy: 0.6143 - val_loss: 1.0091
Epoch 3/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 174s 562ms/step - accuracy: 0.7345 - loss: 0.6963 - val_accuracy: 0.4800 - val_loss: 1.5112
Epoch 4/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 193s 622ms/step - accuracy: 0.7849 - loss: 0.5767 - val_accuracy: 0.7962 - val_loss: 0.5677
Epoch 5/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 194s 625ms/step - accuracy: 0.8052 - loss: 0.5336 - val_accuracy: 0.8257 - val_loss: 0.4437
Epoch 6/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 95s 304ms/step - accuracy: 0.8376 - loss: 0.4514 - val_accuracy: 0.8410 - val_loss: 0.4263
Epoch 7/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 86s 279ms/step - accuracy: 0.8315 - loss: 0.4522 - val_accuracy: 0.8267 - val_loss: 0.4741
66/66 ━━━━━━━━━━━━━━━━━━━━ 8s 112ms/step - accuracy: 0.8024 - loss: 0.5184
Accuracy o

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 100s 319ms/step - accuracy: 0.5085 - loss: 1.3275 - val_accuracy: 0.3324 - val_loss: 2.4431
Epoch 2/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 87s 282ms/step - accuracy: 0.6807 - loss: 0.8559 - val_accuracy: 0.5438 - val_loss: 1.3384
Epoch 3/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 102s 328ms/step - accuracy: 0.7428 - loss: 0.6862 - val_accuracy: 0.6524 - val_loss: 0.9028
Epoch 4/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 101s 327ms/step - accuracy: 0.7572 - loss: 0.6177 - val_accuracy: 0.7467 - val_loss: 0.6160
Epoch 5/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 101s 325ms/step - accuracy: 0.7979 - loss: 0.5302 - val_accuracy: 0.8057 - val_loss: 0.4892
Epoch 6/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 102s 329ms/step - accuracy: 0.8231 - loss: 0.4560 - val_accuracy: 0.8714 - val_loss: 0.3003
Epoch 7/7
307/307 ━━━━━━━━━━━━━━━━━━━━ 101s 327ms/step - accuracy: 0.8378 - loss: 0.4274 - val_accuracy: 0.8505 - val_loss: 0.3763
66/66 ━━━━━━━━━━━━━━━━━━━━ 9s 141ms/step - accuracy: 0.8191 - loss: 0.4314
Accuracy 

In [15]:
#error
results_csv_path = 'experiment_results.csv'
confusion_matrix_folder = 'confusion_matrices'
os.makedirs(confusion_matrix_folder, exist_ok=True)

# Prepare the results DataFrame
columns = ['activation', 'optimizer', 'num_layers', 'neurons', 'accuracy', 'precision', 'recall', 'f1_score']
results_df = pd.DataFrame(columns=columns)

# Run experiments
for activation in activations:
    for optimizer in optimizers:
        
                print(f"Training model with {activation}, {optimizer}, {num_layers} layers, {neurons} neurons, batch size {batch_size}")

                # Build the model (assuming a function build_custom_cnn is defined)
                model = build_custom_cnn(activation=activation, optimizer=optimizer, num_layers=num_layers, neurons=neurons)
                
                # Train the model with callbacks
                model.fit(
                    train_generator,
                    epochs=7,
                    validation_data=validation_generator,
                   
                    verbose=1
                )
                
                # Evaluate the model
                test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)
                print(f"Accuracy of {activation}_{optimizer}_{num_layers}_{neurons} model: {test_accuracy}")

                # Get true and predicted labels
                y_true = test_generator.classes
                y_pred_probs = model.predict(test_generator)
                y_pred = np.argmax(y_pred_probs, axis=1)

                # Calculate precision, recall, and F1 score
                precision = precision_score(y_true, y_pred, average='weighted')
                recall = recall_score(y_true, y_pred, average='weighted')
                f1 = f1_score(y_true, y_pred, average='weighted')

                # Print the classification report
                report = classification_report(y_true, y_pred, output_dict=True)
                print(report)

                # Store the results in DataFrame
                results_df = results_df.concat({
                    'activation': activation,
                    'optimizer': optimizer,
                    'num_layers': num_layers,
                    'neurons': neurons,
                    'accuracy': test_accuracy,
                    'precision': report['accuracy'],  # Use overall accuracy from the report
                    'recall': report['weighted avg']['recall'],
                    'f1_score': report['weighted avg']['f1-score']
                }, ignore_index=True)

                # Generate and save confusion matrix
                cm = confusion_matrix(y_true, y_pred)
                plt.figure(figsize=(10, 7))
                sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=test_generator.class_indices.keys(), yticklabels=test_generator.class_indices.keys())
                plt.ylabel('True label')
                plt.xlabel('Predicted label')
                plt.title(f'Confusion Matrix: {activation}, {optimizer}, Layers: {num_layers}, Neurons: {neurons}')
                plt.savefig(os.path.join(confusion_matrix_folder, f'{activation}_{optimizer}_{neurons}_{num_layers}.png'))
                plt.close()

# Save all experiment results to CSV
results_df.to_csv(results_csv_path, index=False)
print(f"Results saved to {results_csv_path}")

Training model with relu, adam, 4 layers, 128 neurons, batch size 16


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/7


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


 26/307 ━━━━━━━━━━━━━━━━━━━━ 1:24 302ms/step - accuracy: 0.1686 - loss: 3.2504

KeyboardInterrupt: 